# Run network model

In [2]:
import sys
sys.path.append("/home/ubuntu/roaddetection/")
sys.path.append("/media/hh/hd_internal/hh/DSR_Berlin_2018/roaddetection/")

In [7]:
%load_ext autoreload
%autoreload 2

#from keras.layers import merge
from src.models.data import *
from src.models.segnet import *
from src.data.utils import get_tile_prefix
from src.models.metrics_img import auc_roc, auc_pr
#import rasterio.plot as rioplot
import matplotlib
import matplotlib.pyplot as plt
#import matplotlib.image as mpimg

from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, LambdaCallback
from keras.optimizers import *

from pathlib import Path
import os, shutil

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Define and make (if necessary) train/validation/test directories

In [8]:
original_dataset_dir = "../../data/train"
raw_images_path = "../../data/raw/images"
dirs = []

base_dir = "../../data"

train_dir = os.path.join(base_dir, "train")
dirs.append(train_dir)

train_partial_dir = os.path.join(base_dir, "train_partial")
dirs.append(train_partial_dir)

validation_dir = os.path.join(base_dir, "validate")
dirs.append(validation_dir)
test_dir = os.path.join(base_dir, "test")
dirs.append(test_dir)

### Split data up into train/validation/test images

In [9]:
for directory in dirs:
    for file_type in ["sat", "map", "sat_rgb"]:
        target = os.path.join(directory, file_type)
        print(target, ":", len(os.listdir(target)))

print("Done.")

../../data/train/sat : 1372
../../data/train/map : 1372
../../data/train/sat_rgb : 1372
../../data/train_partial/sat : 0
../../data/train_partial/map : 0
../../data/train_partial/sat_rgb : 0
../../data/validate/sat : 280
../../data/validate/map : 280
../../data/validate/sat_rgb : 280
../../data/test/sat : 276
../../data/test/map : 276
../../data/test/sat_rgb : 276
Done.


## User settings

In [10]:
target_size = (512,512)
data_gen_args = dict(
                    data_format="channels_last",
                    horizontal_flip=True, 
                    vertical_flip=True
 )
pretrained_model_fn = None

trained_model_fn = '../../models/segnet_06_12_24_00.hdf5'

batch_size = 5

steps_per_epoch = 280
epochs = 10
validation_steps = 60

optimizer = Adam(lr=1e-4)
loss = 'binary_crossentropy'
loss_weights = None
metrics = ['accuracy', auc_pr]

### Set up ImageDataGenerators for training and validation sets

In [12]:
train_gen = trainGenerator(batch_size,'../../data/train','sat','map',
                        data_gen_args, save_to_dir = None, image_color_mode="rgba", target_size=target_size)

validation_gen = trainGenerator(batch_size,'../../data/validate','sat','map',
                        data_gen_args, save_to_dir = None, image_color_mode="rgba", target_size=target_size)

### Define model, compile, show summary, possibly load weights, define callbacks (including checkpoints)

In [13]:
model = segnet()
model.compile(optimizer=optimizer,
              loss=loss,
              loss_weights=loss_weights,
              metrics=metrics)
model.summary()
if (pretrained_model_fn):
    model.load_weights(pretrained_model_fn)
model_checkpoint = ModelCheckpoint(trained_model_fn, monitor='loss',verbose=1, save_best_only=True)

#Stop training if loss doesn't improve for 2 consecutive epochs
early_stop = EarlyStopping(monitor='loss', min_delta=0, patience=5, verbose=1, mode='auto', baseline=None)

/Users/kprakash/Projects/roaddetection/src/models/segnet.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, activation="relu", padding="valid")`
  conv1 = Conv2D(filter_size, kernel, activation='relu', border_mode='valid')(padding1)
/Users/kprakash/Projects/roaddetection/src/models/segnet.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, 3, activation="relu", padding="valid")`
  conv2 = Conv2D(256, kernel, activation='relu', border_mode='valid')(padding2)
/Users/kprakash/Projects/roaddetection/src/models/segnet.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, 3, activation="relu", padding="valid")`
  conv3 = Conv2D(512, kernel, activation='relu', border_mode='valid')(padding3)
/Users/kprakash/Projects/roaddetection/src/models/segnet.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, 3, padding="valid")`
  conv4 = Conv2D(512, kernel, border_mode='valid')(padding4)
/Users/

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 512, 512, 4)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 514, 514, 4)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 512, 512, 64)      2368      
_________________________________________________________________
batch_normalization_1 (Batch (None, 512, 512, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 256, 256, 64)      0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 258, 258, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 256)     147712    
__________

### Run training

In [14]:
import logging

def get_logger():
    log_fmt = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    logging.basicConfig(level=logging.INFO, format=log_fmt)
    formatter = logging.Formatter(log_fmt)
    fh = logging.FileHandler('../../logs/segnet.log')
    fh.setFormatter(formatter)
    logger = logging.getLogger(__name__)
    logger.addHandler(fh)
    return logger

In [15]:
logger = get_logger()
logging_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: logger.info({'epoch': epoch, 'logs': logs})
)

In [ ]:
history = model.fit_generator(train_gen,
                              steps_per_epoch=steps_per_epoch,
                              epochs=epochs,
                              callbacks=[model_checkpoint, early_stop, logging_callback],
                              validation_data=validation_gen,
                              validation_steps=validation_steps
                             )

Epoch 1/10
Found 280 images belonging to 1 classes.
Found 1372 images belonging to 1 classes.
Found 280 images belonging to 1 classes.
Found 1372 images belonging to 1 classes.


In [ ]:
import matplotlib.pyplot as plt

def plot_history(history):
    plt.plot(history["acc"], label="acc")
    plt.plot(history["val_acc"], label="val_acc")
    plt.legend()
    plt.show()
    plt.close()
    
    plt.plot(history["loss"], label="loss")
    plt.plot(history["val_loss"], label="val_loss")
    plt.legend()
    plt.show()
    plt.close()

#     plt.plot(history["auc_roc"], label="auc_roc")
#     plt.plot(history["val_auc_roc"], label="val_auc_roc")

    plt.plot(history["auc_pr"], label="auc_pr")
    plt.plot(history["val_auc_pr"], label="val_auc_pr")

    plt.legend()
    plt.show()
    plt.savefig("../../logs/unet_borneo_and_harz_05_09_11_11.jpg")
    plt.close()

plot_history(history.history)

In [ ]:
testGene = testGenerator("../../data/test/sat",target_size=(512,512),as_gray=False)
n = 0
for img, name in testGene:
    results = model.predict(img, batch_size=1)
    saveResult("../../data/test/predict", results, name)
    n += 1
    if(n>300):
        break